In [3]:
from finn.util.visualization import showInNetron

showInNetron("/workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/df_model.onnx")

Serving '/workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/df_model.onnx' at http://0.0.0.0:8081


In [4]:
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition

model = ModelWrapper("/workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/df_model.onnx")

for idx, n in enumerate(model.graph.node):
    inst = getCustomOp(n)
    inst.set_nodeattr("partition_id", idx)

model = model.transform(CreateDataflowPartition())
    
model.save("/tmp/fclayer.onnx")

0-------
1-------
2-------
3-------
4-------
5-------
6-------
7-------
8-------
9-------
10-------
11-------
12-------
13-------
14-------
15-------
16-------
17-------
18-------
19-------
20-------
21-------
22-------
23-------
24-------
25-------
26-------
27-------
28-------
29-------
30-------
31-------
32-------
33-------
34-------
35-------
36-------
37-------
38-------
39-------
40-------
41-------
42-------
43-------
44-------
45-------
46-------
47-------
48-------
49-------
50-------
51-------
52-------
53-------
54-------
55-------
56-------
57-------
58-------
59-------
60-------
61-------
62-------
63-------
64-------
65-------
66-------
67-------
68-------
69-------
70-------
71-------
72-------
73-------
74-------
75-------
76-------
77-------
78-------
79-------
80-------
81-------
82-------
83-------
84-------
85-------
86-------
87-------
88-------
89-------
90-------
91-------
92-------
93-------
94-------
95-------
96-------
97-------
98-------
99-------
100-------

In [5]:
model = ModelWrapper("/tmp/fclayer.onnx")

for n in model.graph.node:
    path = getCustomOp(n).get_nodeattr("model")
    model_partition = ModelWrapper(path)
    assert(len(model_partition.graph.node)==1)
    name = model_partition.graph.node[0].name
    #if name=="Vector_Vector_Activate_Batch_20":
    if name=="StreamingFCLayer_Batch_0":
        model_partition.save("/tmp/fclayer_0.onnx")

In [6]:
showInNetron("/tmp/fclayer_0.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/fclayer_0.onnx' at http://0.0.0.0:8081


In [7]:
model = ModelWrapper("/tmp/fclayer_0.onnx")

for n in model.graph.node:
    inst = getCustomOp(n)
    inst.set_nodeattr("code_gen_dir_cppsim","")
    inst.set_nodeattr("executable_path","")
    inst.set_nodeattr("res_hls","")
    inst.set_nodeattr("code_gen_dir_ipgen","")
    inst.set_nodeattr("ipgen_path","")
    inst.set_nodeattr("ip_path","")
    inst.set_nodeattr("ip_vlnv","")
    inst.set_nodeattr("partition_id",0)
    inst.set_nodeattr("cycles_estimate",0)


model.save("/tmp/fclayer_0_cleaned.onnx")

In [8]:
from finn.transformation.general import ApplyConfig

model = ModelWrapper("/tmp/fclayer_0_cleaned.onnx")

model = model.transform(ApplyConfig("/workspace/results/fclayer_0_config.json"))

model.save("/tmp/fclayer_0_config.onnx")

In [9]:
showInNetron("/tmp/fclayer_0_config.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/fclayer_0_config.onnx' at http://0.0.0.0:8081


In [10]:
model = ModelWrapper("/tmp/fclayer_0_cleaned.onnx")

model.save("/workspace/results/fclayer_0.onnx")

In [51]:
model = ModelWrapper("/tmp/streamingfclayer_44_cleaned.onnx")

for n in model.graph.node:
    inst = getCustomOp(n)
    inst.set_nodeattr("PE", 1)
    inst.set_nodeattr("SIMD", 8)
    inst.set_nodeattr("resType", "dsp")
    inst.set_nodeattr("mem_mode", "decoupled")
    inst.set_nodeattr("ram_style", "ultra")
    inst.set_nodeattr("runtime_writeable_weights", 1)

model.save("/tmp/streamingfclayer_44_hls.onnx")

In [45]:
showInNetron("/tmp/streamingfclayer_44_hls.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/streamingfclayer_44_hls.onnx' at http://0.0.0.0:8081


In [ ]:
from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.synth_ooc import SynthOutOfContext

fpgapart = "xcu250-figd2104-2L-e"
clk = 5

model = model.transform(PrepareIP(fpgapart, clk))
model = model.transform(HLSSynthIP())

model.save("/tmp/streamingfclayer_44_hls.onnx")

model = ModelWrapper("/tmp/streamingfclayer_44_hls.onnx")

model = model.transform(CreateStitchedIP(fpgapart, clk))
model = model.transform(SynthOutOfContext(fpgapart, clk))

In [8]:
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp
from finn.transformation.general import ApplyConfig

model = ModelWrapper("/workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/df_model.onnx")

model = model.transform(ApplyConfig("/workspace/results/syn_baseline/folding_config.json"))

for n in model.graph.node:
    if n.op_type=="ConvolutionInputGenerator1D":
        fold_convinpgen = getCustomOp(n).get_nodeattr("SIMD")
        cons = model.find_consumer(n.output[0])
        if cons.op_type=="Vector_Vector_Activate_Batch":
            fold_vvau = getCustomOp(cons).get_nodeattr("PE")
            assert(fold_vvau==fold_convinpgen)
            
    if n.op_type=="StreamingFCLayer_Batch":
        mw = getCustomOp(n).get_nodeattr("MH")
        mh = getCustomOp(n).get_nodeattr("MW")
        pe = getCustomOp(n).get_nodeattr("PE")
        simd = getCustomOp(n).get_nodeattr("SIMD")
        if mh%pe!=0:
            print(n.name)
        if mw%simd!=0:
            print(n.name)

model.save("/tmp/df_model_folded.onnx")

StreamingFCLayer_Batch_93


In [5]:
from finn.util.visualization import showInNetron
showInNetron("/tmp/df_model_folded.onnx")

Serving '/tmp/df_model_folded.onnx' at http://0.0.0.0:8081
